In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import pandas as pd
import wave
import sys
import os
import librosa
import librosa.display
import xgboost as xgb
from  sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import sklearn.naive_bayes as nb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier,GradientBoostingClassifier
from sklearn.neighbors import kd_tree
import seaborn as sn
from sklearn.metrics import confusion_matrix
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.kd_tree module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packag

In [2]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Conv1D, MaxPooling1D, Flatten
from keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.regularizers import l2


from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

Using TensorFlow backend.


In [0]:
import pickle
folder = "/content/drive/My Drive/Respiratory_Sound_Database/Respiratory_Sound_Database/" 
filename = folder + 'w_c_dataset.pickle'
infile = open(filename,'rb')
[sound,sr,lengths,times,labels] = pickle.load(infile)
infile.close()

In [0]:
def split_sounds(sounds,times,labels):
    s=[]
    l=[]
    for i,sound in enumerate(sounds):
        for t,label in zip(times[i],labels[i]):
            s.append(sound[int(t[0]):int(t[1])])
            if label==0:
                a=np.array([1,0,0,0])
            if label==1:
                a=np.array([0,1,0,0])
            if label==2:
                a=np.array([0,0,1,0])    
            if label==3:
                a=np.array([0,0,0,1])
            l.append(a)
    return s,l


In [0]:
[data,label]=split_sounds(sound,times,labels)

In [0]:
import librosa

def extract_feature(cycle_number,X,sample_rate):

    mfcc = librosa.feature.mfcc(y=X, sr=sample_rate,n_mfcc=40)
    mel_spec = librosa.feature.melspectrogram(y=X, sr=sample_rate, n_mels=40)
    ps_db= librosa.power_to_db(mel_spec, ref=np.max)

    mean_mfcc = np.mean(mfcc.T,axis=0)
    mean_ps_db = np.mean(ps_db.T,axis=0)
    std_mfcc = np.std(mfcc.T,axis=0)
    std_ps_db = np.std(ps_db.T,axis=0)
    check = np.hstack((mean_mfcc,std_mfcc,mean_ps_db,std_ps_db))
    return check

In [9]:
dataset = []
for cycle_number,d in enumerate(data):
    a = extract_feature(cycle_number+1,d,sr[0])
    dataset.append(a)
    if(cycle_number%1000 == 0):
        print(cycle_number)



data=np.asarray(dataset)
#data = data.reshape([6898,data.shape[1]*data.shape[2],])
print(data.shape)

label=np.asarray(label)
a=np.zeros(label.shape[0])
for i in range(label.shape[0]):
    for j in range(label.shape[1]):
        if label[i][j]==1:
            a[i]=j

# COnvert to binary for the binary classification problem
#bin_label =np.zeros(label.shape[0])
#for index,l in enumerate(a):
 #   if l == 0:
  #      bin_label[index] = l
   # else:
    #    bin_label[index] = 1

unique, counts = np.unique(a, return_counts=True)

print(np.asarray((unique, counts)).T)

# One-hot encode labels
le = LabelEncoder()
i_labels = le.fit_transform(a)
oh_labels = to_categorical(i_labels) 

0
1000
2000
3000
4000
5000
6000
(6898, 160)
[[0.000e+00 3.642e+03]
 [1.000e+00 1.864e+03]
 [2.000e+00 8.860e+02]
 [3.000e+00 5.060e+02]]


In [0]:
scaler=StandardScaler()
x_train,x_test,y_train,y_test = sklearn.model_selection.train_test_split(dataset,i_labels,test_size=0.2, random_state=42,stratify=i_labels)
scaler.fit(x_train)
x_train=scaler.transform(x_train)
x_test=scaler.transform(x_test)

In [12]:
Cs = [2**(-2),2**(-1), 1,2**(1),2**(2),2**(3),2**(4),2**(5),2**(6),2**(7),2**(8),2**(9),2**(10)]
gamma = [2**(-7),2**(-6),2**(-5),2**(-4),2**(-3),2**(-2),2**(-1),2**(0),2**(1),2**(2),2**(3)]


param_grid = {'C': Cs,  
                'gamma': gamma, 
                'kernel': ['rbf'],
                'decision_function_shape': ['ovr'],
                'class_weight': ['balanced']}  



grid1 = GridSearchCV(SVC(), param_grid,cv=3,n_jobs=-1, verbose = 3) 

# fitting the model for grid search 
grid1.fit(x_train, y_train)

# print best parameter after tuning 
print("Best Parameters after tuning:")
print(grid1.best_params_) 
# print how our model looks after hyper-parameter tuning
print("Best selected Model: ") 
print(grid1.best_estimator_)

grid_predictions = grid1.predict(x_test) 

# print classification report
print("Classification Report: ") 
print(classification_report(y_test, grid_predictions))
print("Classification Accuracy: ")
print(accuracy_score(y_test,grid_predictions))

print("Confusion Matrix is as Follows: ")
print(sklearn.metrics.confusion_matrix(y_test,grid_predictions))

Fitting 3 folds for each of 143 candidates, totalling 429 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed: 22.5min
[Parallel(n_jobs=-1)]: Done 429 out of 429 | elapsed: 34.2min finished


Best Parameters after tuning:
{'C': 16, 'class_weight': 'balanced', 'decision_function_shape': 'ovr', 'gamma': 0.015625, 'kernel': 'rbf'}
Best selected Model: 
SVC(C=16, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.015625, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Classification Report: 
              precision    recall  f1-score   support

           0       0.84      0.85      0.84       729
           1       0.70      0.74      0.72       373
           2       0.74      0.63      0.68       177
           3       0.55      0.51      0.53       101

    accuracy                           0.77      1380
   macro avg       0.70      0.68      0.69      1380
weighted avg       0.77      0.77      0.77      1380

Classification Accuracy: 
0.7666666666666667
Confusion Matrix is as Follows: 
[[618  83  24   4]
 [ 68 276   6  23]
 [ 41   8

In [13]:
# Running XgBoost
import xgboost as xgb

print('Training XGB Classifier from new features:')
xgb_model = xgb.XGBClassifier(max_depth=40,num_class=4, n_estimators=60, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, objective='multi:softmax', eta=0.3, silent=0, subsample=0.8).fit(x_train, y_train)

xgb_pred = xgb_model.predict(x_test)
print(accuracy_score(y_test,xgb_pred))

Training XGB Classifier from new features:
0.7369565217391304
